# Why Pydantic

https://docs.pydantic.dev/2.7/

It's used everywhere 😍 📈

> For a more comprehensive list of open-source projects using Pydantic see the list of dependents on github, or you can find some awesome projects using Pydantic in awesome-pydantic.

https://docs.pydantic.dev/2.7/#who-is-using-pydantic

Interview Samuel on Pybites podcast:
https://www.pybitespodcast.com/14997890/14997890-160-unpacking-pydantic-s-growth-and-the-launch-of-logfire-with-samuel-colvin

## Use cases

- Data Validation: Ensure data conforms to defined types and constraints.
- Data Parsing / Serialization: Parse and convert data from various formats (e.g., JSON) into Python objects.
- Settings Management: Manage configuration and settings in a type-safe manner.
- Data Modeling: Define clear and explicit data models for APIs, databases, etc.

In [1]:
!pip install pydantic


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


# Compared to a dataclass

In [2]:
from dataclasses import dataclass

In [3]:
@dataclass
class User:
    name: str
    age: int

user = User(name="Alice", age=30)
print(user)

User(name='Alice', age=30)


In [4]:
User(name="Alice", age="thirty")  # no built-in validation

User(name='Alice', age='thirty')

# Easy validation with some not too strict types

In [5]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

user = User(name="Alice", age=30)
print(user)

name='Alice' age=30


## Developers love good error messages

In [6]:
user = User(name="Alice", age="thirty")

ValidationError: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [8]:
from pydantic import ValidationError

try:
    user = User(name="Alice", age="thirty")
except ValidationError as ve:
    print(ve)

1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


## Automatic type casting

In [12]:
User(name="Alice", age="30")

User(name='Alice', age=30)

## But Pydantic can go stricter if you want

In [13]:
class User(BaseModel):
    name: str
    age: int

    class Config:
        strict = True
        
User(name="Alice", age=30)


User(name='Alice', age=30)

In [15]:
int("30")

30

In [14]:
User(name="Alice", age="30")

ValidationError: 1 validation error for User
age
  Input should be a valid integer [type=int_type, input_value='30', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type

## You can nest models, making powerful objects

In [26]:
class Address(BaseModel):
    street: str
    city: str

class User(BaseModel):
    name: str
    age: int
    address: Address

user = User(name="Alice", age="30", address={"street": "Main St", "city": "Springfield"})
user.age

30

## Some other useful model methods

In [27]:
user = User(name="Alice", age=30, address={"street": "Main St", "city": "Springfield"})
dir(user)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [28]:
help(User.model_validate)

Help on method model_validate in module pydantic.main:

model_validate(obj: 'Any', *, strict: 'bool | None' = None, from_attributes: 'bool | None' = None, context: 'dict[str, Any] | None' = None) -> 'Model' method of pydantic._internal._model_construction.ModelMetaclass instance
    Validate a pydantic model instance.
    
    Args:
        obj: The object to validate.
        strict: Whether to enforce types strictly.
        from_attributes: Whether to extract data from object attributes.
        context: Additional context to pass to the validator.
    
    Raises:
        ValidationError: If the object could not be validated.
    
    Returns:
        The validated model instance.



In [74]:
# TODO: fix this:
address = Address(street="calle", city="madrid")
User.model_validate({'age': 123, 'name': 'James', 'addres': address})

ValidationError: 1 validation error for User
email
  Field required [type=missing, input_value={'age': 123, 'name': 'Jam...'calle', city='madrid')}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [35]:
from pprint import pp

pp(User.model_json_schema())

{'$defs': {'Address': {'properties': {'street': {'title': 'Street',
                                                 'type': 'string'},
                                      'city': {'title': 'City',
                                               'type': 'string'}},
                       'required': ['street', 'city'],
                       'title': 'Address',
                       'type': 'object'}},
 'properties': {'name': {'title': 'Name', 'type': 'string'},
                'age': {'title': 'Age', 'type': 'integer'},
                'address': {'$ref': '#/$defs/Address'}},
 'required': ['name', 'age', 'address'],
 'title': 'User',
 'type': 'object'}


## Write your own validator

In [43]:
from pydantic import field_validator

class User(BaseModel):
    name: str
    age: int

    @field_validator('age')
    def age_must_be_positive(cls, v):
        if v < 0:
            raise ValueError('age must be positive')
        return v

In [44]:
User(name="Alice", age=30)

User(name='Alice', age=30)

In [45]:
User(name='Alice', age=-1)

ValidationError: 1 validation error for User
age
  Value error, age must be positive [type=value_error, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

In [46]:
from pydantic import Field
from datetime import date, timedelta

class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=10)
    signup_date: date

    @field_validator('signup_date')
    def check_date_not_in_future(cls, v):
        if v > date.today():
            raise ValueError('signup_date cannot be in the future')
        return v

In [49]:
User(username="pybob", signup_date=date.today())

User(username='pybob', signup_date=datetime.date(2024, 7, 9))

In [50]:
User(username="pybob", signup_date=date.today() + timedelta(days=1))

ValidationError: 1 validation error for User
signup_date
  Value error, signup_date cannot be in the future [type=value_error, input_value=datetime.date(2024, 7, 10), input_type=date]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

## Field has this already though

In [52]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., min_length=3)
    age: int = Field(..., ge=18)

In [53]:
User(name="Alice", age=30)

User(name='Alice', age=30)

In [57]:
User(name="Bob", age=16)

ValidationError: 1 validation error for User
age
  Input should be greater than or equal to 18 [type=greater_than_equal, input_value=16, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/greater_than_equal

## Pattern (regex) matching

In [61]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str
    age: int
    # https://docs.pydantic.dev/2.7/api/fields/#pydantic.fields.Field
    email: str = Field(..., pattern='^\\S+@\\S+\\.\\S+$')

User(name="Alice", age=30, email="alice@example.com")

User(name='Alice', age=30, email='alice@example.com')

In [62]:
User(name="Alice", age=30, email="aliceexample.com")

ValidationError: 1 validation error for User
email
  String should match pattern '^\S+@\S+\.\S+$' [type=string_pattern_mismatch, input_value='aliceexample.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/string_pattern_mismatch

## But why regex/re-inventing the wheel? Plugins!

In [63]:
!pip install email-validator


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [65]:
from pydantic import BaseModel, Field, EmailStr

class User(BaseModel):
    name: str
    age: int
    email: EmailStr

User(name='Alice', age=30, email='alice@example.com')


User(name='Alice', age=30, email='alice@example.com')

## Bonus: better error messages

In [66]:
User(name='Alice', age=30, email='aliceexample.com')

ValidationError: 1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='aliceexample.com', input_type=str]

In [67]:
User(name='Alice', age=30, email='alice@example.eu.invalid')

ValidationError: 1 validation error for User
email
  value is not a valid email address: The part after the @-sign is a special-use or reserved name that cannot be used with email. [type=value_error, input_value='alice@example.eu.invalid', input_type=str]

In [68]:
User(name='Alice', age=30, email='alice 1@example.es')

ValidationError: 1 validation error for User
email
  value is not a valid email address: The email address contains invalid characters before the @-sign: SPACE. [type=value_error, input_value='alice 1@example.es', input_type=str]

## Example con* Pydantic types

In [70]:
from pydantic import conlist

class Item(BaseModel):
    id: int
    name: str

class Order(BaseModel):
    items: conlist(Item, min_length=2, max_length=4)

item1 = Item(id=1, name='item1')
item2 = Item(id=2, name='item2')
item3 = Item(id=3, name='item3')
item4 = Item(id=4, name='item4')
item5 = Item(id=5, name='item5')

In [71]:
Order(items=[item1])

ValidationError: 1 validation error for Order
items
  List should have at least 2 items after validation, not 1 [type=too_short, input_value=[Item(id=1, name='item1')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.7/v/too_short

In [72]:
Order(items=[item1, item2, item3, item4, item5])

ValidationError: 1 validation error for Order
items
  List should have at most 4 items after validation, not 5 [type=too_long, input_value=[Item(id=1, name='item1')...tem(id=5, name='item5')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.7/v/too_long

In [73]:
Order(items=[item1, item2, item3])
Order(items=[item1, item2, item3, item4])

Order(items=[Item(id=1, name='item1'), Item(id=2, name='item2'), Item(id=3, name='item3'), Item(id=4, name='item4')])

## FastAPI

Let's quickly look how Pydantic is nicely integrated into FastAPI

## Type checking

I have not used `TypeAdapter` yet, but maybe we can play a bit with mypy and using Pydantic models for static typing?

## v2 performance

Check out PyO3: https://apythonistalearningrust.com/rust-in-python-pyo3-maturin/